In [1]:
from sklearn.externals import joblib
clf = joblib.load('ldaModel.pkl')
print(clf)


LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='batch', learning_offset=10.0,
             max_doc_update_iter=100, max_iter=10, mean_change_tol=0.001,
             n_components=10, n_jobs=1, n_topics=40, perp_tol=0.1,
             random_state=None, topic_word_prior=None,
             total_samples=1000000.0, verbose=0)


In [4]:
clf.transform(["order"])

ValueError: Expected 2D array, got 1D array instead:
array=['order'].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [ ]:
prediction = clf.predict(["hi my mouse is having issue in connectivity"])
print(jsonify({'prediction': list(prediction)}))

In [5]:
# derived from http://scikit-learn.org/stable/auto_examples/applications/topics_extraction_with_nmf_lda.html
# explanations are located there : https://www.linkedin.com/pulse/dissociating-training-predicting-latent-dirichlet-lucien-tardres

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import pickle

# create a blank model
lda = LatentDirichletAllocation()

# load parameters from file
with open ('outfile', 'rb') as fd:
    (features,lda.components_,lda.exp_dirichlet_component_,lda.doc_topic_prior_) = pickle.load(fd)
joblib.load('ldaModel.pkl')
# the dataset to predict on (first two samples were also in the training set so one can compare)
data_samples = ["I like to eat broccoli and bananas.",
                "I ate a banana and spinach smoothie for breakfast.",
                "kittens and dogs are boring"
               ]
# Vectorize the training set using the model features as vocabulary
tf_vectorizer = CountVectorizer(vocabulary=features)
tf = tf_vectorizer.fit_transform(data_samples)

# transform method returns a matrix with one line per document, columns being topics weight
predict = lda.transform(tf)
print(predict)

FileNotFoundError: [Errno 2] No such file or directory: 'outfile'

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.datasets import fetch_20newsgroups
from sklearn.decomposition import NMF, LatentDirichletAllocation

def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print ("Topic %d:" % (topic_idx))
        print (" ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))

dataset = fetch_20newsgroups(shuffle=True, random_state=1, remove=('headers', 'footers', 'quotes'))
documents = dataset.data

no_features = 1000

# NMF is able to use tf-idf
tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2, max_features=no_features, stop_words='english')
tfidf = tfidf_vectorizer.fit_transform(documents)
tfidf_feature_names = tfidf_vectorizer.get_feature_names()

# LDA can only use raw term counts for LDA because it is a probabilistic graphical model
tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2, max_features=no_features, stop_words='english')
tf = tf_vectorizer.fit_transform(documents)
tf_feature_names = tf_vectorizer.get_feature_names()

no_topics = 20

# Run NMF
nmf = NMF(n_components=no_topics, random_state=1, alpha=.1, l1_ratio=.5, init='nndsvd').fit(tfidf)

# Run LDA
lda = LatentDirichletAllocation(n_topics=no_topics, max_iter=5, learning_method='online', learning_offset=50.,random_state=0).fit(tf)

no_top_words = 10
display_topics(nmf, tfidf_feature_names, no_top_words)
display_topics(lda, tf_feature_names, no_top_words)


/anaconda3/lib/python3.6/site-packages/sklearn/decomposition/online_lda.py:294: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)


Topic 0:
people time right did good said say make way government
Topic 1:
window problem using server application screen display motif manager running
Topic 2:
god jesus bible christ faith believe christian christians sin church
Topic 3:
game team year games season players play hockey win league
Topic 4:
new 00 sale 10 price offer shipping condition 20 15
Topic 5:
thanks mail advance hi looking info help information address appreciated
Topic 6:
windows file files dos program version ftp ms directory running
Topic 7:
edu soon cs university ftp internet article email pub david
Topic 8:
key chip clipper encryption keys escrow government public algorithm nsa
Topic 9:
drive scsi drives hard disk ide floppy controller cd mac
Topic 10:
just ll thought tell oh little fine work wanted mean
Topic 11:
does know anybody mean work say doesn help exist program
Topic 12:
card video monitor cards drivers bus vga driver color memory
Topic 13:
like sounds looks look bike sound lot things really thing
To

In [ ]:
# derived from http://scikit-learn.org/stable/auto_examples/applications/topics_extraction_with_nmf_lda.html
# explanations are located there : https://www.linkedin.com/pulse/dissociating-training-predicting-latent-dirichlet-lucien-tardres

from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import pickle

n_features = 50
n_topics = 2

# Training dataset
data_samples = ["I like to eat broccoli and bananas.",
                "I ate a banana and spinach smoothie for breakfast.",
                "Chinchillas and kittens are cute.",
                "My sister adopted a kitten yesterday.",
                "Look at this cute hamster munching on a piece of broccoli."
               ]
# extract fetures and vectorize dataset
tf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=1,
                                max_features=n_features,
                                stop_words='english')
tf = tf_vectorizer.fit_transform(data_samples)

#save features
dic = tf_vectorizer.get_feature_names()

lda = LatentDirichletAllocation(n_topics=n_topics, max_iter=5,
                                learning_method='online',
                                learning_offset=50.,
                                random_state=0)

# train LDA
p1 = lda.fit(tf)

# Save all data necessary for later prediction
model = (dic,lda.components_,lda.exp_dirichlet_component_,lda.doc_topic_prior_)

with open('outfile', 'wb') as fp:
    pickle.dump(model, fp)

In [18]:
# derived from http://scikit-learn.org/stable/auto_examples/applications/topics_extraction_with_nmf_lda.html
# explanations are located there : https://www.linkedin.com/pulse/dissociating-training-predicting-latent-dirichlet-lucien-tardres

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import pickle

# create a blank model
lda = LatentDirichletAllocation()

# load parameters from file
with open ('outfile', 'rb') as fd:
    (features,lda.components_,lda.exp_dirichlet_component_,lda.doc_topic_prior_) = pickle.load(fd)

# the dataset to predict on (first two samples were also in the training set so one can compare)
data_samples = ["who I like to eat  and bananas. what the fuck",
                "I ate a banana and spinach smoothie for breakfast.",
                "I like to eat broccoli and bananas."
               ]
# Vectorize the training set using the model features as vocabulary
tf_vectorizer = TfidfVectorizer(vocabulary=features)
tf = tf_vectorizer.fit_transform(data_samples)

# transform method returns a matrix with one line per document, columns being topics weight
predict = lda.transform(tf)
print(predict)

[[0.30037154 0.69962846]
 [0.37713139 0.62286861]
 [0.31943191 0.68056809]]


In [20]:
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()

In [21]:
pyLDAvis.sklearn.prepare(lda, tf, tf_vectorizer)

/anaconda3/lib/python3.6/site-packages/pyLDAvis/_prepare.py:387: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  topic_term_dists = topic_term_dists.ix[topic_order]


PreparedData(topic_coordinates=            Freq  cluster  topics         x    y
topic                                           
1      66.443888        1       1  0.001433  0.0
0      33.556112        1       2 -0.001433  0.0, topic_info=     Category      Freq         Term     Total  loglift  logprob
term                                                            
14    Default  0.000000     munching  0.000000  20.0000  20.0000
16    Default  0.000000       sister  0.000000  19.0000  19.0000
6     Default  0.000000  chinchillas  0.000000  18.0000  18.0000
2     Default  0.000000       banana  0.000000  17.0000  17.0000
10    Default  0.000000       kitten  0.000000  16.0000  16.0000
13    Default  0.000000         look  0.000000  15.0000  15.0000
7     Default  0.000000         cute  0.000000  14.0000  14.0000
0     Default  0.000000      adopted  0.000000  13.0000  13.0000
5     Default  0.000000     broccoli  0.000000  12.0000  12.0000
17    Default  0.000000     smoothie  0.000000  11.0000  11.0000
9     Default  0.000000      hamster  0.000000  10.0000  10.0000
15    Default  0.000000        piece  0.000000   9.0000   9.0000
8     Default  0.000000          eat  0.000000   8.0000   8.0000
11    Default  0.000000      kittens  0.000000   7.0000   7.0000
4     Default  0.000000    breakfast  0.000000   6.0000   6.0000
3     Default  0.000000      bananas  0.000000   5.0000   5.0000
1     Default  0.000000          ate  0.000000   4.0000   4.0000
19    Default  0.000000    yesterday  0.000000   3.0000   3.0000
18    Default  0.000000      spinach  0.000000   2.0000   2.0000
12    Default  0.000000         like  0.000000   1.0000   1.0000
12     Topic1  0.204257         like  0.278873   0.0974  -2.9633
18     Topic1  0.218909      spinach  0.305810   0.0745  -2.8941
19     Topic1  0.204846    yesterday  0.287802   0.0688  -2.9605
1      Topic1  0.226080          ate  0.322878   0.0524  -2.8618
3      Topic1  0.190031      bananas  0.276287   0.0346  -3.0355
11     Topic1  0.209432      kittens  0.308100   0.0228  -2.9383
4      Topic1  0.196928    breakfast  0.290483   0.0201  -2.9999
8      Topic1  0.196788          eat  0.293955   0.0075  -3.0006
9      Topic1  0.201606      hamster  0.302580   0.0028  -2.9764
17     Topic1  0.211854     smoothie  0.317991   0.0027  -2.9268
5      Topic1  0.204263     broccoli  0.308005  -0.0019  -2.9633
0      Topic1  0.220147      adopted  0.332415  -0.0033  -2.8884
15     Topic1  0.169062        piece  0.257289  -0.0111  -3.1525
7      Topic1  0.199882         cute  0.307206  -0.0210  -2.9850
13     Topic1  0.188905         look  0.292698  -0.0291  -3.0415
10     Topic1  0.198231       kitten  0.308733  -0.0342  -2.9933
2      Topic1  0.186187       banana  0.296597  -0.0568  -3.0560
6      Topic1  0.174315  chinchillas  0.282138  -0.0727  -3.1219
16     Topic1  0.172378       sister  0.283992  -0.0904  -3.1330
14     Topic1  0.180861     munching  0.298500  -0.0922  -3.0850
14     Topic2  0.117639     munching  0.298500   0.1608  -2.8320
16     Topic2  0.111614       sister  0.283992   0.1581  -2.8845
6      Topic2  0.107822  chinchillas  0.282138   0.1300  -2.9191
2      Topic2  0.110410       banana  0.296597   0.1038  -2.8954
10     Topic2  0.110502       kitten  0.308733   0.0645  -2.8946
13     Topic2  0.103793         look  0.292698   0.0552  -2.9572
7      Topic2  0.107324         cute  0.307206   0.0403  -2.9237
15     Topic2  0.088227        piece  0.257289   0.0217  -3.1197
0      Topic2  0.112268      adopted  0.332415   0.0065  -2.8787
5      Topic2  0.103742     broccoli  0.308005   0.0037  -2.9577
17     Topic2  0.106138     smoothie  0.317991  -0.0053  -2.9349
9      Topic2  0.100974      hamster  0.302580  -0.0055  -2.9847
8      Topic2  0.097167          eat  0.293955  -0.0150  -3.0232
4      Topic2  0.093555    breakfast  0.290483  -0.0410  -3.0610
11     Topic2  0.098668      kittens  0.308100  -0.0467  -3.0078
3      Topic2  0.086257      bananas  0.276287  -0.0722 